<a href="https://colab.research.google.com/github/jboiie/SemCom/blob/main/model_extraction/scripts/SemCom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## git

In [1]:
!git clone https://github.com/jboiie/SemCom

fatal: destination path 'SemCom' already exists and is not an empty directory.


In [2]:
%cd SemCom

/content/SemCom


In [ ]:
''' !rm -rf SemCom
!git clone https://github.com/jboiie/SemCom.git '''


# DATASET

## dataset stuff

In [3]:
import os

dataset_path = "model_extraction/data/labeled_triples.json"  # update if needed

if os.path.exists(dataset_path):
    print(f"File exists: {dataset_path}")
    print(f"File size: {os.path.getsize(dataset_path)} bytes\n")
    with open(dataset_path, "r") as f:
        print("First 500 characters of file:\n")
        print(f.read(1000))
else:
    print(f"File NOT found at: {dataset_path}")


File exists: model_extraction/data/labeled_triples.json
File size: 5414 bytes

First 500 characters of file:

[
    {"text": "Paris is the capital of France.", "triples": "Paris|is capital of|France"},
    {"text": "Albert Einstein developed the theory of relativity.", "triples": "Albert Einstein|developed|theory of relativity"},
    {"text": "The Moon revolves around the Earth.", "triples": "Moon|revolves around|Earth"},
    {"text": "Water boils at 100 degrees Celsius.", "triples": "Water|boils at|100 degrees Celsius"},
    {"text": "The Taj Mahal is located in India.", "triples": "Taj Mahal|is located in|India"},
    {"text": "The Nile flows through Egypt.", "triples": "Nile|flows through|Egypt"},
    {"text": "Mount Everest is the highest mountain on Earth.", "triples": "Mount Everest|is highest mountain on|Earth"},
    {"text": "Bananas are rich in potassium.", "triples": "Bananas|are rich in|potassium"},
    {"text": "The Pacific Ocean is the largest ocean on Earth.", "triples": 

In [4]:
from google.colab import files
uploaded = files.upload()

# This lets you upload files interactively.
# After uploading, you can load them as normal.


KeyboardInterrupt: 

In [5]:
dataset_path = "SemCom/model_extraction/data/labeled_triples.json"


## dependencies

In [ ]:
!pip install transformers datasets accelerate


In [ ]:
!pip install -U datasets


In [6]:
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset



In [7]:
import json

## dataset outputs testing

In [ ]:
from transformers import AutoTokenizer, AutoModel

# List of encoder model names to test
model_names = [
    "bert-base-uncased",
    "roberta-base",
    "distilbert-base-uncased"
]

# Loop through the models and load each one
for model_name in model_names:
    print(f"Loading {model_name} ...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    print(f"{model_name} loaded successfully!\n")


In [ ]:
import json

# Replace this path with your actual dataset file path
dataset_path = "model_extraction/data/labeled_triples.json"

# Load the JSON file
with open(dataset_path, "r") as f:
    data = json.load(f)

print(f"Loaded {len(data)} samples")
print("First sample:", data[7])


In [ ]:
from datasets import Dataset

hf_dataset = Dataset.from_list(data)
print(hf_dataset[3])
''' converting dataset into a hugging face dataset '''


## tokenization

In [ ]:
from transformers import AutoTokenizer

# Load the Flan-T5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

# Define the preprocessing function for tokenization
def preprocess_function(batch):
    inputs = batch["text"]
    targets = batch["triples"]

    # Tokenize inputs (for encoder)
    model_inputs = tokenizer(inputs, padding="max_length", max_length=64, truncation=True)

    # Tokenize targets (for decoder)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, padding="max_length", max_length=32, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization function to the entire dataset
tokenized_dataset = hf_dataset.map(preprocess_function, batched=True)

# Look at one tokenized example to verify
print(tokenized_dataset[0])


# MODEL

## intial loading and tokenizing

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import json


In [9]:
# Load your cleaned JSON dataset file with pure JSON array (no variable assignment)
dataset_path = "model_extraction/data/labeled_triples.json"

with open(dataset_path, "r") as f:
    data = json.load(f)

# Convert list of dicts into Hugging Face Dataset object
hf_dataset = Dataset.from_list(data)
print(f"Loaded {len(hf_dataset)} samples")
print(hf_dataset[0])


Loaded 49 samples
{'text': 'Paris is the capital of France.', 'triples': 'Paris|is capital of|France'}


In [10]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [11]:
''' tokenizer functions defined here '''

def preprocess_function(batch):
    inputs = batch["text"]
    targets = batch["triples"]

    # Tokenize the inputs (the encoder input)
    model_inputs = tokenizer(inputs, max_length=64, padding="max_length", truncation=True)

    # Tokenize the targets (the decoder input/label) with special tokenizer context
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=32, padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [12]:
''' applying the tokenizer to the dataset '''

tokenized_dataset = hf_dataset.map(preprocess_function, batched=True)
print(tokenized_dataset[0])


Map:   0%|          | 0/49 [00:00<?, ? examples/s]

{'text': 'Paris is the capital of France.', 'triples': 'Paris|is capital of|France', 'input_ids': [1919, 19, 8, 1784, 13, 1410, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [1919, 9175, 159, 1784, 13, 9175, 371, 5219, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


## model loading

In [13]:
" flan t5 for seq2seq learning"
from transformers import AutoModelForSeq2SeqLM

# Load pre-trained Flan-T5 large model (encoder-decoder)
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## setting arguements

In [ ]:
!pip install --upgrade transformers


In [20]:
"setting training arguements"
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./flan_t5_results",  # where checkpoints and logs go
    eval_steps=500,                  # for example, evaluate every 500 steps
    learning_rate=3e-5,               # reasonable starting LR for fine-tuning
    per_device_train_batch_size=8,   # batch size per device/GPU
    per_device_eval_batch_size=8,
    weight_decay=0.01,               # weight decay for regularization
    save_total_limit=3,              # limit the number of checkpoint files
    num_train_epochs=5,              # total number of epochs
    predict_with_generate=True,      # generate sequences to use for eval metrics
    logging_dir='./logs',            # tensorboard logs directory
    logging_steps=10,                # log every 10 steps
    save_strategy="epoch",           # save model every epoch
    report_to=["none"]  # disables W&B, TensorBoard, etc.

)


In [21]:
from transformers import DataCollatorForSeq2Seq

# Create the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [22]:
from transformers import Seq2SeqTrainer

# Split dataset
split_datasets = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator  # <-- recommended, future compatible
)




## training model and its metrics

In [23]:
trainer.train()

Step,Training Loss
10,23.652800
20,19.427000
30,17.384700


TrainOutput(global_step=30, training_loss=20.154828389485676, metrics={'train_runtime': 1457.38, 'train_samples_per_second': 0.151, 'train_steps_per_second': 0.021, 'total_flos': 63381118648320.0, 'train_loss': 20.154828389485676, 'epoch': 5.0})

In [28]:
"!ls -lah"
!ls -l flan-t5-triple-extraction-final/


total 3062440
-rw-r--r-- 1 root root        773 Aug 28 10:26 config.json
-rw-r--r-- 1 root root        142 Aug 28 10:26 generation_config.json
-rw-r--r-- 1 root root 3132668808 Aug 28 10:28 model.safetensors
-rw-r--r-- 1 root root       2543 Aug 28 10:28 special_tokens_map.json
-rw-r--r-- 1 root root     791656 Aug 28 10:28 spiece.model
-rw-r--r-- 1 root root      20830 Aug 28 10:28 tokenizer_config.json
-rw-r--r-- 1 root root    2422497 Aug 28 10:28 tokenizer.json
-rw-r--r-- 1 root root       5841 Aug 28 10:28 training_args.bin


In [24]:
"saving the model"
trainer.save_model("./flan-t5-triple-extraction-final")


In [29]:
!zip -r flan-t5-triple-extraction-final.zip flan-t5-triple-extraction-final/
"download the model"

  adding: flan-t5-triple-extraction-final/ (stored 0%)
  adding: flan-t5-triple-extraction-final/generation_config.json (deflated 29%)
  adding: flan-t5-triple-extraction-final/model.safetensors (deflated 7%)
  adding: flan-t5-triple-extraction-final/training_args.bin (deflated 53%)
  adding: flan-t5-triple-extraction-final/config.json (deflated 48%)
  adding: flan-t5-triple-extraction-final/tokenizer.json (deflated 74%)
  adding: flan-t5-triple-extraction-final/spiece.model (deflated 48%)
  adding: flan-t5-triple-extraction-final/tokenizer_config.json (deflated 95%)
  adding: flan-t5-triple-extraction-final/special_tokens_map.json (deflated 85%)


'download the model'

In [30]:
from google.colab import files
files.download("flan-t5-triple-extraction-final.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### adding model to git

## won't show on github cus its notebook dosent work the same way